<b><h3>COMPLETE CROP ANALYSIS SYSTEM/h3></b>

In [5]:
pip install transformers timm pillow --quiet

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install --upgrade transformers timm

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
    --------------------------------------- 0.3/11.6 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.6 MB 1.3 MB/s eta 0:00:09
   -- ------------------------------------- 0.8/11.6 MB 1.7 MB/s eta 0:00:07
   --- ------------------------------------ 1.0/11.6 MB 1.2 MB/s eta 0:00:09
   ---- ----------------------------------- 1.3/11.6 MB 1.4 MB/s eta 0:00:08
   ------ --------------------------------- 1.8/11.6 MB 1.6 MB/s eta 0:00:07
   -------- ------------------------------- 2.4/11.6 MB 1.7 MB/s eta 0:00:06
   ---------- ----------------------------- 3.1/11.6 MB 2.0 MB/s eta 0:00:05
   ----------- ---------------------------- 3.4/11.6 MB 1.9 MB/s eta 0:00:05
   ------------- -------------------------- 3.9/11.6 MB 1.9 MB/s eta 0:00:05
   --------------- ------------------------ 4.5/11.6 MB 2.0 MB/s eta 0:00:04
   ---------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [16]:
# ========================================
# WEEK 3: COMPLETE CROP ANALYSIS SYSTEM
# Integrating Week 1 (Data Processing) + Week 2 (ML Model) + Week 3 (Disease Detection)
# ========================================

import io
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import FileUpload, Dropdown, Button, VBox, Output, HTML, HBox
from IPython.display import display, clear_output
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from transformers import pipeline
import warnings
warnings.filterwarnings('ignore')

# Set style for better plots
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print("🚀 WEEK 3: COMPREHENSIVE CROP ANALYSIS SYSTEM")
print("=" * 60)

# ========================================
# 1. INTEGRATED DATA FROM WEEK 1 & 2
# ========================================

# Top 15 crops from your analysis
CROPS = [
    'Rice', 'Wheat', 'Maize', 'Sugarcane', 'Cotton', 
    'Soybean', 'Groundnut', 'Sunflower', 'Mustard', 'Jowar',
    'Bajra', 'Barley', 'Gram', 'Turmeric', 'Onion', 'Tomato'
]

# Top agricultural zones in India (from top 15 states)
ZONES = {
    'Punjab': {
        'Temperature_C': 26, 'Humidity_%': 65, 'Rainfall_mm': 180,
        'optimal_crops': ['Rice', 'Wheat', 'Maize', 'Cotton', 'Sugarcane'],
        'climate_type': 'Semi-arid', 'soil_type': 'Alluvial'
    },
    'Haryana': {
        'Temperature_C': 27, 'Humidity_%': 62, 'Rainfall_mm': 160,
        'optimal_crops': ['Wheat', 'Rice', 'Mustard', 'Gram', 'Barley'],
        'climate_type': 'Semi-arid', 'soil_type': 'Alluvial'
    },
    'Uttar Pradesh': {
        'Temperature_C': 28, 'Humidity_%': 68, 'Rainfall_mm': 150,
        'optimal_crops': ['Rice', 'Wheat', 'Sugarcane', 'Maize', 'Gram'],
        'climate_type': 'Subtropical', 'soil_type': 'Alluvial'
    },
    'Maharashtra': {
        'Temperature_C': 30, 'Humidity_%': 70, 'Rainfall_mm': 120,
        'optimal_crops': ['Cotton', 'Sugarcane', 'Soybean', 'Jowar', 'Onion'],
        'climate_type': 'Tropical', 'soil_type': 'Black Cotton'
    },
    'Karnataka': {
        'Temperature_C': 29, 'Humidity_%': 72, 'Rainfall_mm': 140,
        'optimal_crops': ['Rice', 'Maize', 'Cotton', 'Sunflower', 'Turmeric'],
        'climate_type': 'Tropical', 'soil_type': 'Red Laterite'
    },
    'Tamil Nadu': {
        'Temperature_C': 31, 'Humidity_%': 74, 'Rainfall_mm': 110,
        'optimal_crops': ['Rice', 'Groundnut', 'Cotton', 'Sugarcane', 'Turmeric'],
        'climate_type': 'Tropical', 'soil_type': 'Alluvial'
    }
}

# ========================================
# 2. RECREATE WEEK 2 ML MODEL
# ========================================

def create_synthetic_dataset():
    """Create realistic synthetic dataset based on your Week 1 & 2 work"""
    np.random.seed(42)
    
    data = []
    for _ in range(1000):
        zone = np.random.choice(list(ZONES.keys()))
        crop = np.random.choice(CROPS)
        zone_data = ZONES[zone]
        
        # Add realistic variations
        temp = zone_data['Temperature_C'] + np.random.normal(0, 2)
        humidity = zone_data['Humidity_%'] + np.random.normal(0, 5)
        rainfall = zone_data['Rainfall_mm'] + np.random.normal(0, 20)
        ndvi = np.random.uniform(0.3, 0.9)  # NDVI from satellite data
        
        # Realistic yield calculation based on conditions
        base_yield = 3000
        if crop in zone_data['optimal_crops']:
            base_yield += 1000
        
        # Environmental factors impact
        temp_factor = 1 - abs(temp - 28) * 0.02  # Optimal around 28°C
        humidity_factor = 1 - abs(humidity - 65) * 0.01
        rainfall_factor = min(rainfall / 150, 1.5)  # More rain = better yield up to a point
        ndvi_factor = ndvi * 2  # Higher NDVI = healthier crops
        
        yield_val = base_yield * temp_factor * humidity_factor * rainfall_factor * ndvi_factor
        yield_val = max(yield_val + np.random.normal(0, 200), 500)  # Add noise, min 500
        
        data.append({
            'Zone': zone, 'Crop': crop,
            'Temperature_C': round(temp, 1),
            'Humidity_%': round(humidity, 1),
            'Rainfall_mm': round(rainfall, 1),
            'NDVI_mean': round(ndvi, 3),
            'Yield_kg_per_ha': round(yield_val, 0)
        })
    
    return pd.DataFrame(data)

# Create and train model (from your Week 2 work)
print("📊 Creating synthetic dataset based on your Week 1 & 2 analysis...")
df = create_synthetic_dataset()

# Train model
X = df[['Temperature_C', 'Humidity_%', 'Rainfall_mm', 'NDVI_mean']]
y = df['Yield_kg_per_ha']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf = RandomForestRegressor(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"✅ Model trained successfully!")
print(f"   Mean Absolute Error: {mae:.2f} kg/ha")
print(f"   R² Score: {r2:.3f}")

# ========================================
# 3. DISEASE DETECTION MODEL (Week 3 Addition)
# ========================================

print("\n🤖 Loading plant disease detection model...")
try:
    disease_classifier = pipeline(
        "image-classification",
        model="linkanjarad/mobilenet_v2_1.0_224-plant-disease-identification"
    )
    print("✅ Disease detection model loaded successfully!")
except Exception as e:
    print(f"⚠️ Using mock disease detection: {e}")
    disease_classifier = None

def analyze_plant_health(image_bytes):
    """Analyze plant health from uploaded image"""
    if disease_classifier is None:
        # Mock results for demonstration
        mock_diseases = [
            ("Healthy", 0.89, True),
            ("Leaf Blight", 0.76, False),
            ("Rust Disease", 0.82, False),
            ("Mosaic Virus", 0.71, False)
        ]
        result = np.random.choice(len(mock_diseases))
        disease, confidence, is_healthy = mock_diseases[result]
        return is_healthy, confidence, disease
    
    try:
        img = Image.open(io.BytesIO(image_bytes)).convert("RGB")
        predictions = disease_classifier(img, top_k=1)
        
        top_pred = predictions[0]
        label = top_pred["label"]
        confidence = top_pred["score"]
        
        # Determine if healthy
        is_healthy = "healthy" in label.lower()
        disease_name = label.replace("___", " - ").replace("_", " ").title()
        
        return is_healthy, confidence, disease_name
    except Exception as e:
        return False, 0.5, f"Analysis Error: {str(e)}"

# ========================================
# 4. COMPREHENSIVE ANALYSIS FUNCTIONS
# ========================================

def calculate_yield_potential(zone, crop, health_score, is_healthy):
    """Calculate yield potential based on zone, crop, and health"""
    zone_data = ZONES[zone]
    
    # Base environmental conditions
    features = pd.DataFrame([{
        'Temperature_C': zone_data['Temperature_C'],
        'Humidity_%': zone_data['Humidity_%'],
        'Rainfall_mm': zone_data['Rainfall_mm'],
        'NDVI_mean': health_score * 0.8 if is_healthy else health_score * 0.4
    }])
    
    # Predict yield
    predicted_yield = rf.predict(features)[0]
    
    # Adjust based on crop-zone compatibility
    if crop in zone_data['optimal_crops']:
        predicted_yield *= 1.2  # 20% boost for optimal crops
        suitability = "Excellent"
    else:
        predicted_yield *= 0.8  # 20% reduction for non-optimal crops
        suitability = "Moderate"
    
    # Health impact
    if not is_healthy:
        predicted_yield *= 0.7  # 30% reduction for diseased crops
    
    return predicted_yield, suitability

def generate_recommendations(zone, crop, is_healthy, disease_name):
    """Generate comprehensive recommendations"""
    zone_data = ZONES[zone]
    
    recommendations = []
    
    # Health recommendations
    if is_healthy:
        recommendations.append("✅ Plant appears healthy - maintain current care practices")
        recommendations.append("🔄 Continue regular monitoring for early disease detection")
    else:
        recommendations.append(f"⚠️ Disease detected: {disease_name}")
        recommendations.append("🏥 Consult agricultural extension officer immediately")
        recommendations.append("💊 Consider appropriate fungicide/pesticide treatment")
        recommendations.append("🚰 Improve drainage and reduce moisture if applicable")
    
    # Zone-crop compatibility
    if crop in zone_data['optimal_crops']:
        recommendations.append(f"🎯 {crop} is well-suited for {zone} climate")
    else:
        recommendations.append(f"⚠️ {crop} is not optimal for {zone} - consider switching to:")
        recommendations.extend([f"   • {c}" for c in zone_data['optimal_crops'][:3]])
    
    # Environmental recommendations
    optimal_temp = 28
    optimal_humidity = 65
    optimal_rainfall = 150
    
    current_temp = zone_data['Temperature_C']
    current_humidity = zone_data['Humidity_%']
    current_rainfall = zone_data['Rainfall_mm']
    
    if abs(current_temp - optimal_temp) > 3:
        recommendations.append(f"🌡️ Consider temperature management (current: {current_temp}°C, optimal: ~{optimal_temp}°C)")
    
    if abs(current_humidity - optimal_humidity) > 10:
        recommendations.append(f"💧 Monitor humidity levels (current: {current_humidity}%, optimal: ~{optimal_humidity}%)")
    
    if current_rainfall < 100:
        recommendations.append("🌧️ Consider supplemental irrigation due to low rainfall")
    elif current_rainfall > 200:
        recommendations.append("🌊 Ensure proper drainage to prevent waterlogging")
    
    return recommendations

# ========================================
# 5. USER INTERFACE
# ========================================

# Widgets
crop_dropdown = Dropdown(options=CROPS, description='Crop:', value='Rice')
zone_dropdown = Dropdown(options=list(ZONES.keys()), description='Zone:', value='Punjab')
uploader = FileUpload(accept='.jpg,.png,.jpeg', multiple=False, description="Upload plant image:")
analyze_btn = Button(description="🔬 Complete Analysis", button_style='success', layout={'width': '200px'})
output_area = Output()

def perform_complete_analysis(_):
    """Main analysis function integrating all components"""
    output_area.clear_output()
    
    with output_area:
        if not uploader.value:
            print("❌ Please upload a plant/crop image first!")
            return
        
        # Get inputs
        selected_crop = crop_dropdown.value
        selected_zone = zone_dropdown.value
        
        try:
            # Get uploaded image
            if isinstance(uploader.value, tuple):
                uploaded_file = uploader.value[0]
            else:
                uploaded_file = list(uploader.value.values())[0]
            
            img_bytes = uploaded_file['content']
            
            print("🔄 Performing comprehensive crop analysis...")
            print("=" * 70)
            
            # 1. Disease Detection
            print("🔬 PLANT HEALTH ANALYSIS:")
            is_healthy, confidence, disease_name = analyze_plant_health(img_bytes)
            
            health_status = "HEALTHY ✅" if is_healthy else "DISEASED ⚠️"
            print(f"   Status: {health_status}")
            print(f"   Condition: {disease_name}")
            print(f"   Confidence: {confidence:.1%}")
            print()
            
            # 2. Yield Prediction
            print("🌾 YIELD PREDICTION ANALYSIS:")
            predicted_yield, suitability = calculate_yield_potential(
                selected_zone, selected_crop, confidence, is_healthy
            )
            
            zone_data = ZONES[selected_zone]
            print(f"   Crop: {selected_crop}")
            print(f"   Zone: {selected_zone}")
            print(f"   Zone Suitability: {suitability}")
            print(f"   Predicted Yield: {predicted_yield:.0f} kg/ha")
            print()
            
            # 3. Environmental Conditions
            print("🌍 ENVIRONMENTAL CONDITIONS:")
            print(f"   Temperature: {zone_data['Temperature_C']}°C")
            print(f"   Humidity: {zone_data['Humidity_%']}%")
            print(f"   Rainfall: {zone_data['Rainfall_mm']}mm")
            print(f"   Climate Type: {zone_data['climate_type']}")
            print(f"   Soil Type: {zone_data['soil_type']}")
            print()
            
            # 4. Comprehensive Recommendations
            print("💡 COMPREHENSIVE RECOMMENDATIONS:")
            recommendations = generate_recommendations(selected_zone, selected_crop, is_healthy, disease_name)
            for i, rec in enumerate(recommendations, 1):
                print(f"   {i}. {rec}")
            print()
            
            # 5. Feature Importance (from your Week 2 work)
            print("📊 FEATURE IMPORTANCE ANALYSIS:")
            feature_importance = pd.DataFrame({
                'Feature': X.columns,
                'Importance': rf.feature_importances_
            }).sort_values('Importance', ascending=False)
            
            for _, row in feature_importance.iterrows():
                print(f"   {row['Feature']}: {row['Importance']:.1%}")
            print()
            
            # 6. Climate Resilience Simulation (from Week 2)
            print("🌡️ CLIMATE RESILIENCE SIMULATION:")
            base_features = pd.DataFrame([{
                'Temperature_C': zone_data['Temperature_C'],
                'Humidity_%': zone_data['Humidity_%'],
                'Rainfall_mm': zone_data['Rainfall_mm'],
                'NDVI_mean': confidence * 0.8 if is_healthy else confidence * 0.4
            }])
            
            # Simulate climate stress
            stress_features = base_features.copy()
            stress_features['Temperature_C'] *= 1.1  # +10% temperature
            stress_features['Rainfall_mm'] *= 0.85   # -15% rainfall
            
            normal_yield = rf.predict(base_features)[0]
            stress_yield = rf.predict(stress_features)[0]
            yield_loss = ((normal_yield - stress_yield) / normal_yield) * 100
            
            print(f"   Normal conditions: {normal_yield:.0f} kg/ha")
            print(f"   Climate stress: {stress_yield:.0f} kg/ha")
            print(f"   Potential yield loss: {yield_loss:.1f}%")
            print()
            
            print("=" * 70)
            print("📸 ANALYZED IMAGE:")
            
            # Display the uploaded image
            img = Image.open(io.BytesIO(img_bytes))
            img.thumbnail((400, 400))
            display(img)
            
            # Create visualization
            create_analysis_visualization(feature_importance, selected_zone, selected_crop, 
                                        predicted_yield, is_healthy)
            
        except Exception as e:
            print(f"❌ Error during analysis: {e}")
            import traceback
            traceback.print_exc()

def create_analysis_visualization(feature_importance, zone, crop, yield_pred, is_healthy):
    """Create comprehensive visualization dashboard"""
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle(f'Crop Analysis Dashboard: {crop} in {zone}', fontsize=16, fontweight='bold')
    
    # 1. Feature Importance
    sns.barplot(data=feature_importance, x='Importance', y='Feature', ax=ax1, palette='viridis')
    ax1.set_title('Feature Importance for Yield Prediction')
    ax1.set_xlabel('Relative Importance')
    
    # 2. Zone Comparison
    zone_yields = []
    zones = list(ZONES.keys())
    for z in zones:
        zd = ZONES[z]
        features = pd.DataFrame([{
            'Temperature_C': zd['Temperature_C'],
            'Humidity_%': zd['Humidity_%'],
            'Rainfall_mm': zd['Rainfall_mm'],
            'NDVI_mean': 0.7
        }])
        zy = rf.predict(features)[0]
        zone_yields.append(zy)
    
    colors = ['green' if z == zone else 'lightblue' for z in zones]
    ax2.bar(zones, zone_yields, color=colors)
    ax2.set_title(f'Yield Potential Across Zones for {crop}')
    ax2.set_ylabel('Yield (kg/ha)')
    ax2.tick_params(axis='x', rotation=45)
    
    # 3. Health Status
    health_colors = ['green', 'red']
    health_labels = ['Healthy', 'Diseased']
    health_values = [1, 0] if is_healthy else [0, 1]
    ax3.pie(health_values, labels=health_labels, colors=health_colors, 
            autopct='%1.0f%%', startangle=90)
    ax3.set_title('Plant Health Status')
    
    # 4. Environmental Conditions Radar (simplified as bar chart)
    zone_data = ZONES[zone]
    conditions = ['Temperature', 'Humidity', 'Rainfall']
    values = [zone_data['Temperature_C']/35*100, zone_data['Humidity_%'], 
              zone_data['Rainfall_mm']/200*100]
    
    ax4.bar(conditions, values, color=['orange', 'blue', 'green'])
    ax4.set_title('Environmental Conditions (Normalized)')
    ax4.set_ylabel('Normalized Value (%)')
    ax4.set_ylim(0, 100)
    
    plt.tight_layout()
    plt.show()

# Connect button to analysis function
analyze_btn.on_click(perform_complete_analysis)

# ========================================
# 6. MAIN INTERFACE
# ========================================

# Header
header_html = HTML("""
<div style="background: linear-gradient(90deg, #4CAF50, #45a049); padding: 20px; border-radius: 10px; margin-bottom: 20px;">
    <h1 style="color: white; text-align: center; margin: 0;">
        🌱 COMPREHENSIVE CROP ANALYSIS SYSTEM
    </h1>
    <h3 style="color: white; text-align: center; margin: 5px 0 0 0;">
        Week 3: Integration of Disease Detection + Yield Prediction + Environmental Analysis
    </h3>
</div>
""")

instructions_html = HTML("""
<div style="background: #f0f8ff; padding: 15px; border-radius: 8px; border-left: 4px solid #4CAF50;">
    <h3>📋 Instructions:</h3>
    <ol>
        <li><strong>Select your crop</strong> from the dropdown (15 major Indian crops available)</li>
        <li><strong>Choose your zone</strong> from top agricultural regions in India</li>
        <li><strong>Upload a clear image</strong> of your crop/plant leaf (JPG/PNG format)</li>
        <li><strong>Click "Complete Analysis"</strong> to get comprehensive results including:
            <ul>
                <li>🔬 Disease detection and health scoring</li>
                <li>🌾 Yield prediction based on environmental conditions</li>
                <li>🌍 Zone-crop suitability analysis</li>
                <li>💡 Actionable recommendations</li>
                <li>📊 Data visualizations and climate resilience insights</li>
            </ul>
        </li>
    </ol>
</div>
""")

# Create the complete interface
input_section = VBox([
    HBox([crop_dropdown, zone_dropdown]),
    uploader,
    analyze_btn
])

main_interface = VBox([
    header_html,
    instructions_html,
    input_section,
    output_area
])

# Display the interface
print("\n🎯 WEEK 3 PROJECT: READY FOR COMPREHENSIVE ANALYSIS!")
print("This system integrates:")
print("  ✅ Week 1: Data preprocessing & environmental data")
print("  ✅ Week 2: ML model training & yield prediction") 
print("  ✅ Week 3: Disease detection & complete analysis")
print("\n" + "="*70)

display(main_interface)

🚀 WEEK 3: COMPREHENSIVE CROP ANALYSIS SYSTEM
📊 Creating synthetic dataset based on your Week 1 & 2 analysis...
✅ Model trained successfully!
   Mean Absolute Error: 505.43 kg/ha
   R² Score: 0.792

🤖 Loading plant disease detection model...


Device set to use cpu


✅ Disease detection model loaded successfully!

🎯 WEEK 3 PROJECT: READY FOR COMPREHENSIVE ANALYSIS!
This system integrates:
  ✅ Week 1: Data preprocessing & environmental data
  ✅ Week 2: ML model training & yield prediction
  ✅ Week 3: Disease detection & complete analysis

